# Ruokavalion optimointi - mitä ruokia ja kuinka paljon pitää syödä, jotta päivittäiset saantisuositukset täyttyvät?

##### optimoinnin tulos lista kertoimia (satoina grammoina), joiden mukainen määrä kutakin elintarviketta tulee syödä

ravintosisältötiedot peräisin Finelin versiosta 20

rajoituksia:
* kutakin ravintoainetta oltava (vähintään?) suositusten mukainen määrä
* kaikki kertoimet 0 - 900 (?)
* kertoimien summa alle 5000
* mahdollisimman pieni määrä nollasta poikkeavia kertoimia
* ruokien yhteenlaskettu energiamäärä 2000 kcal (* 4,184) - tai mahdollisimman pieni?

Työnkulku:
* luetaan 5 csv-tiedostoa: component_value_utf-8.csv, eufdname_FI_utf-8.csv, food_utf-8.csv, saantisuositus_2014.csv ja component_utf-8.csv -> read_files()
* poistetaan food-taulusta lisäravinteet, laihdutusvalmisteet, äidinmaidonkorvikkeet yms. -> filter_fuclass()
* poistetaaan food-taulusta myös keinotekoisesti makeutetut virvoitusjuomat
* poistetaan component_value-taulusta rivit, jotka liittyvät edellisessä vaiheessa poistettuihin valmisteisiin -> filter_components_by_food()
* tehdään component_value-taululle pivot: ennen jokaisella ruoka + ravinne -parilla oli oma rivi, nyt kunkin ruuan tiedot kerätään yhdelle riville ja ravinteet ovat omissa sarakkeissaan
* suodatetaan component_value-taulu niin, että siinä ovat mukana vain ne ravinteet, jotka halutaan mukaan optimointiin; lähtökohtaisesti ne, joille on saantisuositus olemassa, mutta jossain vaiheessa käyttäjä voi mahdollisesti lisätä/poistaa ravinteita -> limit_components()
* luodaan scipy.optimize.linprog()-funktion parametrit A, b, c, bounds
  - A-matriisi: listojen lista, jossa kukin alilista on yhden ravinteen määrät eri ruuissa -> create_matrix_a_eq()
  - b-vektori, joka sisältää saantisuositukset
  - c-vektori, joka sisältää eri ruokien energiamäärät -> get_single_component_values()
  - bounds: yhden ruoan määrän yläraja (tällä hetkellä 8 eli 800 g)
  
#TODO: miten saa painettua minimaaliset kertoimet nollaan?

In [169]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

In [170]:
# pois laskuista jätettävät elintarviketyypit (FUCLASS):
# lastenruoat, äidinmaidonkorvikkeet, ateriankorvikkeet, lisäravinteet ja 
# keinotekoisesti makeutetut virvoitusjuomat

omitted_food_types = ('BABYFTOT', 
'BABMEATD',
'BABFISHD',
'BABMILPO',
'BABWATPO',
'BABFRUB',
'BABVEGE',
'BABMIFRU',
'BABOTHER',
'MMILK',
'INFMILK',
'CASMILK',
'PREMILK',
'SOYMILK',
'WHEYMILK',
'AMINMILK',
'SPECTOT',
'SPECSUPP',
'MEALREP',
'SPORTFOO',
'SPECFOOD',
'DRINKART')

In [171]:
def read_files(path:str) -> tuple:
    """This function reads the following csv files and returns 
    a tuple of pandas data structures: 
    component_value_utf-8.csv
    eufdname_FI_utf-8.csv
    food_utf-8.csv
    saantisuositus_2014.csv

    The function also removes data for various supplements,
    since the target is to look at real foods.

    Args:
        path (str): absolute path to csv files

    Returns:
        tuple: Returns a tuple of pandas data structures with the 
        data from the csv files. (component_value, eufdname, food,
        saantisuositus)
    """

    component_value = pd.read_csv(path + "component_value_utf-8.csv", sep=";", keep_default_na=False)
    eufdname = pd.read_csv(path + "eufdname_FI_utf-8.csv", sep=";")
    food = pd.read_csv(path + "food_utf-8.csv", sep=";")
    saantisuositus = pd.read_csv(path + "saantisuositus_2014.csv", sep=";", header=None, names=["EUFDNAME", "name", "mnuori", "maikuinen", "mkeski", "miäkäs", "mvanha", "npieni","nnuori", "naikuinen", "nkeski", "niäkäs", "nvanha"])
    component = pd.read_csv(path + "component_utf-8.csv", sep=";", dtype={'EUFDNAME':str}, keep_default_na=False)

    return component_value, eufdname, food, saantisuositus, component

In [172]:
component_value, eufdname, food, saantisuositus, component = read_files('/home/pomo/Asiakirjat/Kurssit/Taitotalo_Python-ohjelmoija/python/portfolio/fineli_20/')
print(component_value.head())
print(eufdname.head())
print(food.head())
print(saantisuositus.head())
print(component.head())
print(component_value['EUFDNAME'][45:50])
print(component_value['BESTLOC'][45:50])

   FOODID EUFDNAME  BESTLOC ACQTYPE METHTYPE METHIND
0       1    ENERC  1698.30       S        S  MI0107
1       1      FAT        0       F        E  MIR003
2       1   CHOAVL   99.900       S        S  MI0181
3       1   CHOCDF   99.880       S        S  MI0131
4       1     PROT        0       L        P  MIR003
  THSCODE                              DESCRIPT LANG
0   ENERC               energia, laskennallinen   FI
1  XENERC  energia, laskennallinen ilman kuitua   FI
2     FAT                                 rasva   FI
3  CHOAVL               hiilihydraatti imeytyvä   FI
4  CHOCDF             hiilihydraatti erotuksena   FI
   FOODID                  FOODNAME FOODTYPE PROCESS  EDPORT   IGCLASS  \
0       1                    SOKERI     FOOD     IND     100  SUGARSYR   
1       2  FRUKTOOSI, HEDELMÄSOKERI     FOOD     IND     100  SUGARSYR   
2       3                  SIIRAPPI     FOOD     IND     100  SUGARSYR   
3       4                    HUNAJA     FOOD     RAW     100  SUGARS

In [196]:
# poistetaan rivit, joissa ei ole BESTLOC-arvoa

component_value = component_value[component_value.BESTLOC != ""]

# muutetaan object-tyyppiset luvut liukuluvuiksi
component_value['BESTLOC'] = component_value['BESTLOC'].astype(float)
print(component_value['BESTLOC'][45:50])

45    0.01
46    0.01
47    0.10
48    0.25
49    0.00
Name: BESTLOC, dtype: float64


In [174]:
# get the full list of components (=nutrients)
component_list = sorted(component['EUFDNAME'].tolist())
component_list

['ALC',
 'ASH',
 'CA',
 'CAROTENS',
 'CARTB',
 'CHOAVL',
 'CHOCDF',
 'CHOLE',
 'CR',
 'CU',
 'ENERC',
 'F16D0T',
 'F18D1T',
 'F18D2CN6',
 'F18D3N3',
 'F20D4N6',
 'F20D5N3',
 'F22D6N3',
 'FACIDCTG',
 'FAFRE',
 'FAMCIS',
 'FAPU',
 'FAPUN3',
 'FAPUN6',
 'FAS18',
 'FASAT',
 'FAT',
 'FATRN',
 'FD',
 'FE',
 'FIBC',
 'FIBINS',
 'FIBT',
 'FOL',
 'FRUS',
 'GALS',
 'GLUS',
 'ID',
 'K',
 'LACS',
 'MALS',
 'MG',
 'MN',
 'MO',
 'MYRIC',
 'NA',
 'NACL',
 'NIA',
 'NIAEQ',
 'NT',
 'OA',
 'P',
 'PROT',
 'PSACNCS',
 'QUERCE',
 'RETOL',
 'RIBF',
 'SE',
 'STARCH',
 'STERT',
 'SUCS',
 'SUGAR',
 'SUGOH',
 'THIA',
 'TRP',
 'VITA',
 'VITB12',
 'VITC',
 'VITD',
 'VITE',
 'VITK',
 'VITPYRID',
 'WATER',
 'ZN']

In [175]:
def filter_fuclass(dataframe:pd.DataFrame, fuclass_to_remove:tuple) -> pd.DataFrame:
    """Take a pandas dataframe with food data and remove
    lines where the FUCLASS is one of those specified in 
    fuclass_to_remove. Returns the cleaned dataframe.

    Args:
        dataframe (pandas_df): A pandas dataframe with food names and food use class (FUCLASS)
        fuclass_to_remove (tuple): A tuple of FUCLASS values to use for filtering out unwanted data from the dataframe

    Returns:
        pandas_df: The pandas dataframe minus the lines with specified FUCLASS values
    """
    for food_type in fuclass_to_remove:
        dataframe = dataframe[dataframe.FUCLASS != food_type]

    return dataframe

In [176]:
# puuttuvien ravintoarvotietojen käsittely

def transpose_component_value(dataframe:pd.DataFrame) -> pd.DataFrame:
    """Takes a pandas dataframe (component_value) where each row represents
    the amount of one nutrient in one food (e.g. calcium in milk). 
    Transposes the dataframe so that there is one row per food and one column per nutrient.
    Fills in any missing component values with zeroes in the appropriate column.
    The ACQTYPE, METHTYPE, and METHIND columns are dropped.

    Args:
        dataframe (pandas_df): A pandas dataframe with information on how much of each nutrient foods contain

    Returns:
        pandas_df: The pandas dataframe modified so that it has one row per food and one column per nutrient.
    """
    df = dataframe.drop(columns=['ACQTYPE','METHTYPE', 'METHIND'])

    new_df = df.pivot_table(values='BESTLOC', index='FOODID', columns='EUFDNAME', fill_value=0)
    
    return new_df
    
df = transpose_component_value(component_value)
df.head()

EUFDNAME,ALC,ASH,CA,CAROTENS,CARTB,CHOAVL,CHOCDF,CHOLE,CR,CU,...,TRP,VITA,VITB12,VITC,VITD,VITE,VITK,VITPYRID,WATER,ZN
FOODID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.02,0.4,0.0,0.0,99.9,99.88,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.01
2,0.0,0.00,0.1,0.0,0.0,99.8,99.70,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.3,0.01
3,0.0,1.10,74.0,0.0,0.0,74.1,79.30,0.0,5.0,0.240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,19.5,0.13
4,0.0,0.18,5.0,0.0,0.0,80.8,81.32,0.0,5.0,0.030,...,0.0,0.0,0.0,4.0,0.0,0.0,0.02,0.0,18.0,0.10
5,0.0,0.90,7.0,0.0,0.0,87.0,88.60,0.0,26.0,0.175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.0,0.29


In [177]:
def filter_components_by_food(component_value:pd.DataFrame, food:pd.DataFrame) -> pd.DataFrame:
    """Takes two pandas dataframes: component_value and food. 
    Removes unnecessary rows from component_value 
    to only include rows that have FOODID values found in food.
    If the food dataframe was previously filtered with filter_fuclass(),
    we don't want to drag along rows related to foods that are no longer in the 
    food dataframe.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        food (pandas_df): A dataframe with foods

    Returns:
        pandas_df: A filtered version of component_value
    """
    food_ids = food['FOODID'].tolist()
    new_component_value = component_value[component_value.FOODID.isin(food_ids)]
    
    return new_component_value

In [178]:
# vanha versio, poistetaan
'''def old_limit_components(component_value:pd.DataFrame, nutrient_list:list) -> pd.DataFrame:
    """Takes a pandas dataframe and a list of nutrients to include. 
    Returns a filtered version of the dataframe that only 
    includes rows that have one of the EUFDNAME values found in the list.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of nutrient codes

    Returns:
        pandas_df: A filtered version of component_value
    """
    new_component_value = component_value[component_value.EUFDNAME.isin(nutrient_list)]
    
    return new_component_value
'''

'def old_limit_components(component_value:pd.DataFrame, nutrient_list:list) -> pd.DataFrame:\n    """Takes a pandas dataframe and a list of nutrients to include. \n    Returns a filtered version of the dataframe that only \n    includes rows that have one of the EUFDNAME values found in the list.\n\n    Args:\n        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain\n        nutrient_list (list): A list of nutrient codes\n\n    Returns:\n        pandas_df: A filtered version of component_value\n    """\n    new_component_value = component_value[component_value.EUFDNAME.isin(nutrient_list)]\n    \n    return new_component_value\n'

In [179]:
def limit_components(component_value:pd.DataFrame, nutrient_list:list) -> pd.DataFrame:
    """Takes a pandas dataframe and a list of nutrients to include. 
    Returns a filtered version of the dataframe that only 
    includes the columns for the EUFDNAME values found in the list.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of nutrient codes

    Returns:
        pandas_df: A filtered version of component_value
    """
    new_component_value = component_value[nutrient_list]
    
    return new_component_value

In [180]:
def get_single_component_values(component_value:pd.DataFrame, component="ENERC") -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and the EUFDNAME of a single nutrient. 
    Returns the values of a specified component for the foods included.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        component (string): the EUFDNAME of a nutrient

    Returns:
        list: A list of the values of the specified component (default: ENERC) for the foods
    """
    specified_only = component_value[component_value.EUFDNAME == component]
    
    return specified_only['BESTLOC'].tolist()

In [182]:
# uusi versio funktiosta uudella, pivotoidulla component_value-taululla
def create_matrix_a_eq(component_value:pd.DataFrame, nutrient_list:list) -> list:
    """Takes a pandas dataframe with the nutrient compositions of foods and
    a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        component_value (pandas_df): A dataframe with information on how much of each nutrient foods contain
        nutrient_list (list): A list of EUFDNAMEs

    Returns:
        list of lists: The A_eq matrix where each row is the amounts of one nutrient in the various foods.
    """
    A = []
    for nutrient in nutrient_list:
        A.append(component_value[nutrient].tolist())
        
    return A

In [183]:
def create_rda_list(rda:pd.DataFrame, nutrient_list:list, target_group:str="nkeski") -> list:
    """Takes a pandas dataframe with the recommended daily allowances for various nutrients
    and a list of EUFDNAMEs of nutrients. 
    Returns the A_eq matrix (list of lists) for use in the scipy.optimize.linprog function.

    Args:
        rda (pandas_df): A dataframe with rda values for nutrients
        nutrient_list (list): A list of EUFDNAMEs
        target_group (str): The age & sex combination of the target group (default: middle-aged women)

    Returns:
        list: The b constraint vector where each value is the rda of a nutrient.
    """
    specified_only = rda[rda.EUFDNAME.isin(nutrient_list)]
    
    return specified_only[target_group].tolist()

In [184]:
print(len(food.index))
testi = food.loc[food['FOODID']<24990]
print(len(testi.index))
testi.head()


4238
1378


,FOODID,FOODNAME,FOODTYPE,PROCESS,EDPORT,IGCLASS,IGCLASSP,FUCLASS,FUCLASSP
0,1,SOKERI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
1,2,"FRUKTOOSI, HEDELMÄSOKERI",FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
2,3,SIIRAPPI,FOOD,IND,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
3,4,HUNAJA,FOOD,RAW,100,SUGARSYR,SUGARTOT,SUGADD,SUGARTOT
4,5,"MAKEINEN, KARAMELLI",FOOD,IND,100,SWEET,SUGARTOT,SWEET,SUGARTOT


In [185]:
#filter food
filtered_food = filter_fuclass(food, omitted_food_types)
print(len(filtered_food.index))

# lisäksi poistetaan makeutusaineet
filtered_food = filtered_food[~filtered_food["FOODNAME"].str.contains("MAKEUTUSAINE")]
filtered_food = filtered_food[~filtered_food["FOODNAME"].str.contains("KARPPISOKERI")]
print(len(filtered_food.index))


4139
4128


In [186]:
#filter component_value by foodid
print(len(component_value))
filtered_comp_val = filter_components_by_food(component_value, filtered_food)
print(len(filtered_comp_val))
filtered_comp_val.tail()

307819
300448


,FOODID,EUFDNAME,BESTLOC,ACQTYPE,METHTYPE,METHIND
307829,35887,CHOLE,323.57,S,R,MI0002
307830,35887,STERT,11.43,S,R,MI0002
307831,35887,TRP,486.20,S,R,MI0002
307832,35887,MYRIC,0.00,S,R,MI0002
307833,35887,QUERCE,0.00,S,R,MI0002


In [187]:
# pivot component_value
pivoted_component_value = transpose_component_value(filtered_comp_val)
pivoted_component_value.head()

EUFDNAME,ALC,ASH,CA,CAROTENS,CARTB,CHOAVL,CHOCDF,CHOLE,CR,CU,...,TRP,VITA,VITB12,VITC,VITD,VITE,VITK,VITPYRID,WATER,ZN
FOODID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.02,0.4,0.0,0.0,99.9,99.88,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.01
2,0.0,0.00,0.1,0.0,0.0,99.8,99.70,0.0,1.0,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.3,0.01
3,0.0,1.10,74.0,0.0,0.0,74.1,79.30,0.0,5.0,0.240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,19.5,0.13
4,0.0,0.18,5.0,0.0,0.0,80.8,81.32,0.0,5.0,0.030,...,0.0,0.0,0.0,4.0,0.0,0.0,0.02,0.0,18.0,0.10
5,0.0,0.90,7.0,0.0,0.0,87.0,88.60,0.0,26.0,0.175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,10.0,0.29


In [188]:
# filter component value by nutrient -- tarvitaanko tätä vaihetta? A-matriisiinhan tulevat kuitenkin vain määritetyt ravinteet

reduced_component_value = limit_components(pivoted_component_value, ['VITA', 'VITB12', 'ZN', 'NACL'])
#print(len(filtered_comp_val2))
reduced_component_value.head()

EUFDNAME,VITA,VITB12,ZN,NACL
FOODID,,,,
1,0.0,0.0,0.01,0.25
2,0.0,0.0,0.01,0.25
3,0.0,0.0,0.13,127.40
4,0.0,0.0,0.10,5.61
5,0.0,0.0,0.29,254.80


In [189]:
c = pivoted_component_value['ENERC'].tolist()
print(len(c))
print(c[:10])

4122
[1698.3, 1696.6, 1261.4, 1382.1, 1487.5, 2270.15, 1358.84, 1375.56, 1294.8, 2033.21]


In [190]:
# testi ensin pienellä määrällä ravinteita

A = create_matrix_a_eq(pivoted_component_value, ['VITA', 'VITB12', 'ZN', 'NACL'])
print(len(A[0]))
print(len(A))
print(type(A[0][0]))
print(type(A))

4122
4
<class 'float'>
<class 'list'>


In [191]:
b = create_rda_list(saantisuositus, nutrient_list=['VITA', 'VITB12', 'ZN', 'NACL'], target_group="nkeski")
print(b)

[700.0, 2.0, 7.0, 5000.0]


In [197]:
res = linprog(c, A_eq=A, b_eq=b, bounds=(0, 8))
res

     con: array([-5.51279982e-08, -3.29372973e-10, -4.27323066e-10, -3.59608748e-07])
     fun: 115.39836924550593
 message: 'Optimization terminated successfully.'
     nit: 28
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([2.39536756e-14, 2.39776796e-14, 3.23002529e-14, ...,
       4.83136831e-14, 3.89338039e-13, 2.27142484e-14])

In [198]:
foodid_list = pivoted_component_value.index.tolist()
print(len(foodid_list))
print(len(res.x))
ids_and_amounts = sorted(zip( res.x, foodid_list), reverse=True)
ids_and_amounts[:20]

4122
4122


[(7.9999999994762705, 915),
 (3.9999999999997, 32772),
 (3.9999999999997, 32771),
 (3.9999999999997, 923),
 (1.4006972444916705, 34228),
 (0.6690989783075474, 863),
 (0.04036236522203193, 33151),
 (0.031514383025868675, 788),
 (4.751216412173509e-07, 922),
 (2.316361783923394e-07, 921),
 (1.0999290002694234e-10, 79),
 (4.815054927962486e-11, 901),
 (4.597371674583812e-11, 937),
 (3.496496444614393e-11, 31160),
 (1.2968145512184521e-11, 36),
 (1.1373777379258801e-11, 33428),
 (7.096128020575996e-12, 35555),
 (3.926378315689184e-12, 350),
 (3.903192714568022e-12, 930),
 (3.670254936461712e-12, 30)]

In [201]:
# koko saantisuosituslistalla pois lukien ENERC

rda_minus_enerc = saantisuositus['EUFDNAME'].tolist()[:-1]

A = create_matrix_a_eq(pivoted_component_value, rda_minus_enerc)
b = create_rda_list(saantisuositus, nutrient_list=rda_minus_enerc, target_group="nkeski")
res = linprog(c, A_eq=A, b_eq=b, bounds=(0, 8))
res

     con: array([-2.62427875e-08, -1.94480876e-10, -2.47036169e-10, -2.12669882e-11,
       -2.76343393e-11, -5.73429304e-10, -3.05524495e-11, -4.91553465e-09,
       -1.56769708e-10, -1.11283782e-09, -1.37400775e-08, -2.43597924e-08,
       -4.78421498e-08, -5.61135494e-09, -2.41474396e-10, -2.03238315e-10,
       -1.65198633e-10, -2.46284628e-08, -1.84134308e-09, -4.22076596e-10,
       -1.27744215e-09, -1.46970081e-09, -1.71186912e-07])
     fun: 2224.0774420012276
 message: 'Optimization terminated successfully.'
     nit: 44
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1.29075217e-13, 1.29168680e-13, 1.66410721e-13, ...,
       2.25198729e-14, 2.53905838e-13, 5.46493893e-14])

In [203]:
ids_and_amounts = sorted(zip( res.x, foodid_list), reverse=True)
ids_and_amounts[:20]

[(7.999999999987998, 923),
 (7.999999997867615, 915),
 (6.73435398735487, 33334),
 (6.190275516951755, 901),
 (3.556246258071541, 11069),
 (3.5465957963119634, 921),
 (1.9520447125161102, 32771),
 (1.2549490978387523, 35586),
 (1.1063394581314971, 310),
 (1.0424764646482256, 32770),
 (0.9342024265645238, 34989),
 (0.7820496153110785, 459),
 (0.3498578758957243, 34240),
 (0.33486134020626307, 805),
 (0.29310953898191006, 863),
 (0.25173296831235276, 794),
 (0.2141807218719508, 930),
 (0.15309849987529014, 26),
 (0.10579980623766601, 31488),
 (0.019983095970720177, 812)]